<a href="https://colab.research.google.com/github/eyaler/workshop/blob/master/Hebrew_word_embeddings_and_graphs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U gensim
from gensim.models import KeyedVectors, fasttext
import os

if not os.path.exists('/content/cc.he.300.bin'):
  !wget --no-check-certificate -nc https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.he.300.bin.gz
  !gzip -d /content/cc.he.300.bin.gz

fast_model = fasttext.load_facebook_vectors('/content/cc.he.300.bin')


Requirement already up-to-date: gensim in /usr/local/lib/python3.6/dist-packages (3.8.3)


In [ ]:
print(fast_model.most_similar('חתול'))
print(fast_model.most_similar('חתול', restrict_vocab=100000))

In [ ]:
print(fast_model.most_similar('שונה'))
print(fast_model.most_similar('שונה', restrict_vocab=100000))

In [ ]:
print(fast_model.most_similar(positive=['אישה','מלך'], negative=['גבר']))
print(fast_model.most_similar(positive=['אישה','מלך'], negative=['גבר'], restrict_vocab=100000))

In [ ]:
print(fast_model.most_similar(positive=['אשה','מלך'], negative=['גבר']))
print(fast_model.most_similar(positive=['אשה','מלך'], negative=['גבר'], restrict_vocab=100000))

In [ ]:
print(fast_model.most_similar(positive=['אישה','מלך'], negative=['איש']))
print(fast_model.most_similar(positive=['אישה','מלך'], negative=['איש'], restrict_vocab=100000))

In [ ]:
print(fast_model.most_similar(positive=['אשה','מלך'], negative=['איש']))
print(fast_model.most_similar(positive=['אשה','מלך'], negative=['איש'], restrict_vocab=100000))

In [ ]:
print(fast_model.doesnt_match(['עצוב','שמח','מרוצה','מבסוט']))

In [ ]:
print(fast_model.doesnt_match(['עט','אופניים','ספר','עיתון']))

In [ ]:
print(fast_model.doesnt_match(['עט','אוטו','ספר','עיתון']))

In [ ]:
print(fast_model.doesnt_match(['טוש','אופנוע','ספר','עיתון']))

In [ ]:
restrict_vocab = 10000
model = KeyedVectors(len(fast_model.vectors[0]))
model.add(fast_model.index2word[:restrict_vocab], fast_model.vectors[:restrict_vocab])
fast_model = model

In [ ]:
#https://anotherdatum.com/word-morph.html
top_k_nbs = 10
graph1 = {word: fast_model.most_similar(word, topn=restrict_vocab, restrict_vocab=restrict_vocab) for word in fast_model.index2word[:restrict_vocab]}
graph1['חתול'][:top_k_nbs]

In [ ]:
!pip install editdistance
import editdistance
from collections import defaultdict
graph1a = defaultdict(list)
for word in graph1:
  for nb,_ in graph1[word]:
    if editdistance.eval(nb,word)>=min_dist:
      graph1a[word].append(nb)
    if len(graph1a[word])==top_k_nbs:
      break
graph1a['חתול']

In [ ]:
from collections import defaultdict
graph1b = defaultdict(list)
for word in fast_model.index2word[:restrict_vocab]:
  for nb,_ in fast_model.most_similar(word, topn=restrict_vocab, restrict_vocab=restrict_vocab):
    if all(editdistance.eval(nb,nb1)>=min_dist for nb1 in graph1b[word]+[word]):
      graph1b[word].append(nb)
    if len(graph1b[word])==top_k_nbs:
      break
graph1b['חתול']

In [ ]:
# https://imada.sdu.dk/~zimek/publications/SSDBM2010/SNN-SSDBM2010-preprint.pdf
from collections import defaultdict, Counter
graph2 = defaultdict(Counter)
for word, nbs in graph1.items():
    nbs_plus_word = nbs + [(word,None)]
    for i, (nb1,_) in enumerate(nbs_plus_word[:-1]):
        for nb2,_ in nbs_plus_word[i+1:]:
          if editdistance.eval(nb1,nb2)>=min_dist:
            graph2[nb1][nb2] += 1
            graph2[nb2][nb1] += 1
graph2 = {k: sorted(v.items(), key=lambda x: (-x[1],x[0])) for k,v in graph2.items() if len(v)>0}
graph2['חתול']

In [ ]:
sorted(graph2.items(), key=lambda x: (len(x[1]),x[0])) #highest degree nodes